In [ ]:
while all([weighted_FLUX[ii][jj] <= percent*sum(indexed_EO_sources_FLUX[ii][jj])\
              for ii in range(len(data['data'])) for jj in range(len(indexed_EO_sources_FLUX[ii]))]):

In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.colorbar import Colorbar
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
import matplotlib.mlab as mlab

import scipy
from scipy import io
from scipy.stats import iqr, norm
from scipy.stats.kde import gaussian_kde

from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel

import glob
import re

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import io
import glob

def collector(path):
    '''Collects all IDL save data from a given path and stores each file as an element in a list.'''
    
    filenames = glob.glob(path)
    
    data = {'data':[scipy.io.readsav(filenames[i],python_dict=True) \
            for i in range(len(filenames))],'filenames':filenames}
    return data

def separator(data):
    '''Compiles data into separate lists of extended and point sources'''

    point_data = [[data['data'][i]['source_array'][j] \
        for j in range(len(data['data'][i]['source_array'])) \
        if data['data'][i]['source_array'][j][-2] is None ] \
    for i in range(len(data['data']))]
    
    extended_data = [[data['data'][i]['source_array'][j] \
        for j in range(len(data['data'][i]['source_array'])) \
        if data['data'][i]['source_array'][j][-2] is not None ] \
    for i in range(len(data['data'])) ]

    return {'extsources':extended_data,'psources':point_data}


def pixelate(ra_zoom, dec_zoom, n_bins, ra_total, dec_total, flux_total):
    import numpy as np
    #Check to see which dimension is larger so that a square in ra,dec can 
    #be returned
    if (ra_zoom[1]-ra_zoom[0]) > (dec_zoom[1]-dec_zoom[0]):
        zoom = ra_zoom
    else:
        zoom = dec_zoom

    #Find the size of the bins using the largest dimension and the num of bins
    binsize = (zoom[1]-zoom[0])/n_bins

    #Create arrays for ra and dec that give the left side of each pixel
    spacing_dec = ((ra_zoom[1]-ra_zoom[0]) - (dec_zoom[1]-dec_zoom[0]))/2.
    spacing_ra = ((dec_zoom[1]-dec_zoom[0])-(ra_zoom[1]-ra_zoom[0]))/2.
    
    ra_bin_array = (np.array(range(n_bins)) * binsize) + ra_zoom[0]
    dec_bin_array = (np.array(range(n_bins)) * binsize) + dec_zoom[0]
    
    if (ra_zoom[1]-ra_zoom[0]) > (dec_zoom[1]-dec_zoom[0]):
        dec_bin_array = (np.array(range(n_bins)) * binsize) + dec_zoom[0] - spacing_dec
    if (ra_zoom[1]-ra_zoom[0]) < (dec_zoom[1]-dec_zoom[0]):
        ra_bin_array = (np.array(range(n_bins)) * binsize) + ra_zoom[0] - spacing_ra
        
    #Create an empty array of pixels to be filled in the for loops
    pixels = np.zeros((len(ra_bin_array),len(dec_bin_array)))

    #Histogram components into ra bins
    ra_histogram = np.digitize(ra_total,ra_bin_array)
    ###print ra_histogram

    #Begin for loop over both dimensions of pixels, starting with ra
    for bin_i in range(len(ra_bin_array) - 2):
        ###print range(len(ra_bin_array) -2
        ###print "bin_i",bin_i
        #Find the indices that fall into the current ra bin slice
        ra_inds = np.where(ra_histogram == bin_i)
        ###print "rainds", ra_inds[0]
        ###print "lenrainds", len(ra_inds[0])

        #Go to next for cycle if no indices fall into current ra bin slice
        if len(ra_inds[0]) == 0:
            continue

        #Histogram components that fall into the current ra bin slice by dec
        dec_histogram = np.digitize(dec_total[ra_inds],dec_bin_array)
        #Begin for loop by dec over ra bin slice
        for bin_j in range(len(dec_bin_array) -2):
            
            #Find the indicies that fall into the current dec bin
            dec_inds = np.where(dec_histogram == bin_j)

            #Go to next for cycle if no indices fall into current dec bin			
            if len(dec_inds[0]) == 0:
                continue
            #Sum the flux components that fall into current ra/dec bin
            pixels[bin_i,bin_j] = np.sum(flux_total[ra_inds[0][dec_inds][0]])

    #Find the pixel centers in ra/dec for plotting purposes
    ra_pixel_centers = (np.arange(n_bins) * binsize) + ra_zoom[0] + binsize/2.
    dec_pixel_centers = (np.arange(n_bins) * binsize) + dec_zoom[0] + binsize/2.

    return pixels, ra_pixel_centers, dec_pixel_centers

In [118]:
data = collector('New_Source_Arrays/1130784064_source_array.sav')

In [136]:
def keeper(original_list, old_values, new_value):
    print "old_values:", old_values
    print "new_value:", new_value
    new_list = [i for i
                in original_list
                if i not in old_values]
    new_list.append(new_value)
    return new_list

def seeker(data):
    separated = separator(data)

    indexed_EO_sources_RA = [[[separated['extsources'][i][j]['EXTEND']['RA'][k] \
        for k in range(len(separated['extsources'][i][j]['EXTEND']['RA']))] \
        for j in range(len(separated['extsources'][i]))] \
        for i in range(len(separated['extsources'])) ]

    indexed_EO_sources_DEC = [[[separated['extsources'][i][j]['EXTEND']['DEC'][k] \
        for k in range(len(separated['extsources'][i][j]['EXTEND']['DEC']))] \
        for j in range(len(separated['extsources'][i]))] \
        for i in range(len(separated['extsources'])) ]

    indexed_EO_sources_FLUX = [[[separated['extsources'][i][j]['EXTEND']['FLUX'][k]['I'][0] \
        for k in range(len(separated['extsources'][i][j]['EXTEND']['FLUX'])) ] \
        for j in range(len(separated['extsources'][i]))] \
        for i in range(len(separated['extsources'])) ]
    
    indexed_EO_sources_XX = [[[separated['extsources'][i][j]['EXTEND']['FLUX'][k]['XX'][0] \
        for k in range(len(separated['extsources'][i][j]['EXTEND']['FLUX'])) ] \
        for j in range(len(separated['extsources'][i]))] \
        for i in range(len(separated['extsources'])) ]
    
    indexed_EO_sources_YY = [[[separated['extsources'][i][j]['EXTEND']['FLUX'][k]['YY'][0] \
        for k in range(len(separated['extsources'][i][j]['EXTEND']['FLUX'])) ] \
        for j in range(len(separated['extsources'][i]))] \
        for i in range(len(separated['extsources'])) ]
    
    for i in range(len(data['data'])):
        for j in range(len(indexed_EO_sources_RA[i])):
            for k in range(len(indexed_EO_sources_RA[i][j])):
                if indexed_EO_sources_RA[i][j][k] > 180:
                    indexed_EO_sources_RA[i][j][k] -= 360
                    
    return {'indexed_EO_sources_RA': indexed_EO_sources_RA, 'indexed_EO_sources_DEC': indexed_EO_sources_DEC, \
            'indexed_EO_sources_FLUX': indexed_EO_sources_FLUX}

                    
def chaser(indexed_EO_sources_RA, indexed_EO_sources_DEC, indexed_EO_sources_FLUX, radius=200, n=0):
    print "chaser n:", n
    brightest_indices = [indexed_EO_sources_FLUX[i][j].index(max(indexed_EO_sources_FLUX[i][j][:-n or None]))\
                         for i in range(len(indexed_EO_sources_FLUX))\
                         for j in range(len(indexed_EO_sources_FLUX[i]))]

    # Finds RA, DEC, and FLUX values for the brightest components of each EO,
    # as well as all neighboring components within a user-defined radius.
    #
    brightest_RA = [[[indexed_EO_sources_RA[i][j][k]\
        for k in range(len(indexed_EO_sources_RA[i][j]))\
        if np.sqrt((indexed_EO_sources_RA[i][j][k] - indexed_EO_sources_RA[i][j][brightest_indices[j]])**2\
            + (indexed_EO_sources_DEC[i][j][k] - indexed_EO_sources_DEC[i][j][brightest_indices[j]])**2)\
            < radius/3600.]\
        for j in range(len(indexed_EO_sources_RA[i]))]\
        for i in range(len(indexed_EO_sources_RA))]
    #print "brightest RA:", len(brightest_RA[0][0]), brightest_RA[0][0]
    brightest_DEC = [[[indexed_EO_sources_DEC[i][j][k]\
        for k in range(len(indexed_EO_sources_DEC[i][j]))\
        if np.sqrt((indexed_EO_sources_RA[i][j][k] - indexed_EO_sources_RA[i][j][brightest_indices[j]])**2\
            + (indexed_EO_sources_DEC[i][j][k] - indexed_EO_sources_DEC[i][j][brightest_indices[j]])**2)\
            < radius/3600.]\
        for j in range(len(indexed_EO_sources_DEC[i]))]\
        for i in range(len(indexed_EO_sources_DEC))]
    #print "brightest DEC:", len(brightest_DEC[0][0]), brightest_DEC[0][0]
    brightest_FLUX = [[[indexed_EO_sources_FLUX[i][j][k]\
        for k in range(len(indexed_EO_sources_FLUX[i][j]))\
        if np.sqrt((indexed_EO_sources_RA[i][j][k] - indexed_EO_sources_RA[i][j][brightest_indices[j]])**2\
            + (indexed_EO_sources_DEC[i][j][k] - indexed_EO_sources_DEC[i][j][brightest_indices[j]])**2)\
            < radius/3600.]\
        for j in range(len(indexed_EO_sources_FLUX[i]))]\
        for i in range(len(indexed_EO_sources_FLUX))]
    #print "brightest FLUX:", len(brightest_FLUX[0][0]), brightest_FLUX[0][0]

    return {'brightest_RA': brightest_RA, 'brightest_DEC': brightest_DEC, 'brightest_FLUX': brightest_FLUX}


def beater(brightest_RA,brightest_DEC,brightest_FLUX):
    
    weighted_RA = [[np.average(brightest_RA[i][j], weights=brightest_FLUX[i][j])\
                    for j in range(len(brightest_FLUX[i]))]\
                   for i in range(len(brightest_RA))]

    weighted_DEC = [[np.average(brightest_DEC[i][j], weights=brightest_FLUX[i][j])\
                     for j in range(len(brightest_FLUX[i]))]\
                    for i in range(len(brightest_DEC))]

    weighted_FLUX = [[sum(brightest_FLUX[i][j])\
                      for j in range(len(brightest_FLUX[i]))]\
                     for i in range(len(brightest_FLUX))]    
    
    return {'weighted_RA': weighted_RA, 'weighted_DEC': weighted_DEC, 'weighted_FLUX': weighted_FLUX}


def modeler(data, percent = .05):
    
    sought = seeker(data)

    indexed_EO_sources_RA = sought['indexed_EO_sources_RA']
    indexed_EO_sources_DEC = sought['indexed_EO_sources_DEC']
    indexed_EO_sources_FLUX = sought['indexed_EO_sources_FLUX']
    weighted_FLUX = [[1e5,0]]
    m = 0
    
    for i in range(len(data['data'])):
        for j in range(len(indexed_EO_sources_FLUX[i])):
            while weighted_FLUX[i][j] >= percent*sum(indexed_EO_sources_FLUX[i][j]):
                print "i:", i, "j:", j
                i, j = i, j
                chased = chaser(indexed_EO_sources_RA, indexed_EO_sources_DEC, indexed_EO_sources_FLUX, n=m)
                brightest_RA = chased['brightest_RA']
                brightest_DEC = chased['brightest_DEC']
                brightest_FLUX = chased['brightest_FLUX']

                weighted = beater(brightest_RA,brightest_DEC,brightest_FLUX)
                weighted_RA = weighted['weighted_RA']
                weighted_DEC = weighted['weighted_DEC']
                weighted_FLUX = weighted['weighted_FLUX']
                
                print "RA_0:", len(indexed_EO_sources_RA[i][j])
                print "DEC_0:", len(indexed_EO_sources_DEC[i][j])
                print "flux_0:", len(indexed_EO_sources_FLUX[i][j])
                if weighted_FLUX[i][j] >= percent*sum(indexed_EO_sources_FLUX[i][j]):
                    indexed_EO_sources_RA[i][j] = keeper(indexed_EO_sources_RA[i][j],brightest_RA[i][j],weighted_RA[i][j])
                    indexed_EO_sources_DEC[i][j] = keeper(indexed_EO_sources_DEC[i][j],brightest_DEC[i][j],weighted_DEC[i][j])
                    indexed_EO_sources_FLUX[i][j] = keeper(indexed_EO_sources_FLUX[i][j],brightest_FLUX[i][j],weighted_FLUX[i][j])
                print "RA_f:", len(indexed_EO_sources_RA[i][j])
                print "DEC_f:", len(indexed_EO_sources_DEC[i][j])
                print "flux_f:", len(indexed_EO_sources_FLUX[i][j])
                
                m += 1
            
            


In [137]:
modeler(data)

i: 0 j: 1
chaser n: 0
RA_0: 53
DEC_0: 53
flux_0: 53
old_values: [67.423912, 67.423943, 67.423996, 67.424049, 67.42408, 67.424126, 67.424156, 67.424187, 67.424232, 67.424271, 67.424355, 67.424431, 67.424561, 67.424629, 67.424622, 67.424644, 67.424606, 67.424423, 67.4244, 67.42421, 67.42411, 67.42424, 67.423889, 67.423691, 67.423492, 67.423271, 67.422829, 67.421837, 67.420616, 67.428429, 67.418663, 67.409134, 67.398033, 67.398033, 67.430809, 67.433952, 67.436165, 67.436661, 67.436607, 67.436104, 67.436966, 67.436928]
new_value: 67.4241
old_values: [-36.512577, -36.512566, -36.512608, -36.512661, -36.512749, -36.512745, -36.512783, -36.512802, -36.512852, -36.512947, -36.513035, -36.513092, -36.513195, -36.513191, -36.513283, -36.513344, -36.513359, -36.513416, -36.513466, -36.513729, -36.513779, -36.513897, -36.514168, -36.514301, -36.514439, -36.514587, -36.515015, -36.515305, -36.515369, -36.514584, -36.513828, -36.512943, -36.501225, -36.501225, -36.516273, -36.516968, -36.517963, -36

ValueError: max() arg is an empty sequence